In [7]:
from google.colab import drive
drive.mount('/content/drive')

# TEST_DIR = '/content/drive/My Drive/brain training/'
TEST_DIR = '/content/drive/My Drive/Alzheimer\'s Research/brain training/Results/GreyWhiteMasks/'
TRUTH_DIR = '/content/drive/My Drive/Alzheimer\'s Research/Ground Truth Accuracy Calculation/ground_truth_images/'
SAVE_FILENAME = 'AccuracyResults.csv' # Under TEST_DIR
SAVE_FILENAME_LATEX = 'AccuracyResults.txt' # Under TEST_DIR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
import glob
import numpy as np
from tqdm import tqdm
import re
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

# Stores expected image dimensions
IMAGEDIMNORM = np.array(
        [[55296, 47616], 
        [73728, 47616], 
        [66048, 46080], 
        [69120, 36864], 
        [59904, 43008], 
        [56832, 39936], 
        [62976, 47616], 
        [58368, 44544], 
        [50688, 47616], 
        [58368, 47616], 
        [55296, 46080], 
        [72192, 47616], 
        [69120, 47616], 
        [49152, 43008], 
        [50688, 36864], 
        [55296, 46080], 
        [55296, 41472], 
        [56832, 43008]], dtype='int')
downscale = 128
IMAGEDIMORIG = np.array(
        [[53784, 47177], 
        [73704, 46200], 
        [65736, 45178], 
        [67728, 36459], 
        [59760, 41787], 
        [55776, 39849], 
        [61752, 46410], 
        [57768, 43740], 
        [49800, 47520], 
        [57768, 47481], 
        [53784, 45142], 
        [71712, 47426], 
        [67728, 47200], 
        [47808, 41668], 
        [49800, 36131], 
        [53784, 45548], 
        [53784, 41073], 
        [55776, 41634]], dtype='int')

# Get image names
image_names = glob.glob(TRUTH_DIR + "*Grey.png")
image_names = [imagename.split('/')[-1] for imagename in image_names]
image_names = [imagename.split('-Grey')[0] for imagename in image_names]
image_names = sorted(image_names)
print(image_names)

['NA3777-02_AB', 'NA4077-02_AB', 'NA4092-02_AB', 'NA4107-02_AB', 'NA4160-02_AB', 'NA4195-02_AB', 'NA4256-02_AB', 'NA4299-02_AB', 'NA4391-02_AB', 'NA4450-02_AB', 'NA4463-02_AB', 'NA4471-02_AB', 'NA4553-02_AB', 'NA4626-02_AB', 'NA4672-02_AB', 'NA4675-02_AB', 'NA4691-02_AB', 'NA4695-02_AB']


#### Power analysis for GM and WM separately (Pixel Accuracy, IoU, F1)
https://online.stat.psu.edu/stat414/node/306/

In [9]:
# Per-voxel accuracy difference of a simulated image
dBack = np.zeros(len(image_names))
dGM = np.zeros(len(image_names))
dWM = np.zeros(len(image_names))

# IoU and F1
IoUBack = np.zeros(len(image_names))
IoUGM = np.zeros(len(image_names))
IoUWM = np.zeros(len(image_names))
F1Back = np.zeros(len(image_names))
F1GM = np.zeros(len(image_names))
F1WM = np.zeros(len(image_names))

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
  # Filename for testing and ground truth images
  grey_img_name = imagename + "-Grey.png"
  white_img_name = imagename + "-White.png"
  back_img_name = imagename + "-Background.png"

  ##### Loading Background Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + back_img_name))
  truth_img = Image.open(TRUTH_DIR + back_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img) * 1

  # Check dimensions
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' background dimension WRONG!!!')

  # Update tqdm descriptions
  t.set_description_str("Image {}, Test ({}), Ground Truth ({})".format(
      imagename, str(test_img.shape), str(truth_img.shape)) )
  t.refresh()
  t.write("", end=' ')

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Per-voxel accuracy difference of Background, 0 (match), 1 (unmatch)
  dBack[i] = np.sum(np.abs(test_img - truth_img)) / (width * height)
  # Calculate IoU and F1 score
  IoUBack[i] = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
  F1Back[i] = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

  ##### Loading Grey Matter Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + grey_img_name))
  truth_img = Image.open(TRUTH_DIR + grey_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img) * 1

  # Check dimensions
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' grey dimension WRONG!!!')

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Per-voxel accuracy difference of GM, 0 (match), 1 (unmatch)
  dGM[i] = np.sum(np.abs(test_img - truth_img)) / (width * height)
  # Calculate IoU and F1 score
  IoUGM[i] = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
  F1GM[i] = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

  ##### Loading White Matter Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + white_img_name))
  truth_img = Image.open(TRUTH_DIR + white_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img) * 1

  # Check dimensions
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' white dimension WRONG!!!')

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Per-voxel accuracy difference of WM, 0 (match), 1 (unmatch)
  dWM[i] = np.sum(np.abs(test_img - truth_img)) / (width * height)
  # Calculate IoU and F1 score
  IoUWM[i] = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
  F1WM[i] = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

  t.update()
t.close()

# Convert accuracy difference to 1 (match), 0 (unmatch)
dBack = 1 - dBack
dGM = 1 - dGM
dWM = 1 - dWM

print('\nDone!')

Image NA3777-02_AB, Test ((372, 432)), Ground Truth ((372, 432)):   0%|          | 0/18 [00:07<?, ?it/s]

Image NA4077-02_AB, Test ((372, 576)), Ground Truth ((372, 576)):   6%|▌         | 1/18 [00:21<04:11, 14.81s/it]

Image NA4092-02_AB, Test ((360, 516)), Ground Truth ((360, 516)):  11%|█         | 2/18 [00:36<04:07, 15.45s/it]

Image NA4107-02_AB, Test ((288, 540)), Ground Truth ((288, 540)):  17%|█▋        | 3/18 [00:48<03:40, 14.72s/it]

Image NA4160-02_AB, Test ((336, 468)), Ground Truth ((336, 468)):  22%|██▏       | 4/18 [00:58<03:08, 13.45s/it]

...

Image NA4675-02_AB, Test ((360, 432)), Ground Truth ((360, 432)):  83%|████████▎ | 15/18 [02:59<00:30, 10.25s/it]

Image NA4691-02_AB, Test ((324, 432)), Ground Truth ((324, 432)):  89%|████████▉ | 16/18 [03:09<00:20, 10.32s/it]

Image NA4695-02_AB, Test ((336, 444)), Ground Truth ((336, 444)):  94%|█████████▍| 17/18 [03:19<00:10, 10.02s/it]

Image NA4695-02_AB, Test ((336, 444)), Ground Truth ((336, 444)): 100%|██████████| 18/18 [03:25<00:00,  9.98s/it]


Done!


In [18]:
import scipy.stats

# Significance threshold (acceptable Type I error)
alpha = 0.05
# Calculate z values
z_alpha = scipy.stats.norm(0, 1).ppf(1 - alpha/2)   # two-tailed
# Margin or error in sampled mean
marginErrorBack = 0.05; marginErrorGM = 0.05; marginErrorWM = 0.05;
marginErrorIoUBack = 0.1; marginErrorIoUGM = 0.1; marginErrorIoUWM = 0.1;
marginErrorF1Back = 0.1; marginErrorF1GM = 0.1; marginErrorF1WM = 0.1;

#################### Pixel Accuracy ####################
# Average per-image difference
avgdBack = np.mean(dBack)
avgdGM = np.mean(dGM)
avgdWM = np.mean(dWM)
# Estimate Per-image difference variance
sigdBack = np.sum(np.square(dBack - avgdBack)) / (len(image_names) - 1)
sigdGM = np.sum(np.square(dGM - avgdGM)) / (len(image_names) - 1)
sigdWM = np.sum(np.square(dWM - avgdWM)) / (len(image_names) - 1)

# Estimate sample size
n_dBack = (z_alpha * np.sqrt(sigdBack) / marginErrorBack) ** 2
n_dGM = (z_alpha * np.sqrt(sigdGM) / marginErrorGM) ** 2
n_dWM = (z_alpha * np.sqrt(sigdWM) / marginErrorWM) ** 2

#################### IoU and F1 ####################
# Average IoU and F1
avgIoUBack = np.mean(IoUBack)
avgIoUGM = np.mean(IoUGM)
avgIoUWM = np.mean(IoUWM)
avgF1Back = np.mean(F1Back)
avgF1GM = np.mean(F1GM)
avgF1WM = np.mean(F1WM)
# Estimate IoU and F1 variances
sigIoUBack = np.sum(np.square(IoUBack - avgIoUBack)) / (len(image_names) - 1)
sigIoUGM = np.sum(np.square(IoUGM - avgIoUGM)) / (len(image_names) - 1)
sigIoUWM = np.sum(np.square(IoUWM - avgIoUWM)) / (len(image_names) - 1)
sigF1Back = np.sum(np.square(F1Back - avgF1Back)) / (len(image_names) - 1)
sigF1GM = np.sum(np.square(F1GM - avgF1GM)) / (len(image_names) - 1)
sigF1WM = np.sum(np.square(F1WM - avgF1WM)) / (len(image_names) - 1)

# Estimate sample size
n_IoUBack = (z_alpha * np.sqrt(sigIoUBack) / marginErrorIoUBack) ** 2
n_IoUGM = (z_alpha * np.sqrt(sigIoUGM) / marginErrorIoUGM) ** 2
n_IoUWM = (z_alpha * np.sqrt(sigIoUWM) / marginErrorIoUWM) ** 2
n_F1Back = (z_alpha * np.sqrt(sigF1Back) / marginErrorF1Back) ** 2
n_F1GM = (z_alpha * np.sqrt(sigF1GM) / marginErrorF1GM) ** 2
n_F1WM = (z_alpha * np.sqrt(sigF1WM) / marginErrorF1WM) ** 2

print('alpha: %.4f' % (alpha))
print('z_alpha: %.4f' % (z_alpha))
print('#################### Pixel Accuracy ####################')
print('dBack'); print(dBack); print('')
print('dGM'); print(dGM); print('')
print('dWM'); print(dWM); print('')
print('avgdBack: %.4f \t avgdGM: %.4f \t avgdWM: %.4f' % (avgdBack, avgdGM, avgdWM))
print('sigdBack: %.4f \t sigdGM: %.4f \t sigdWM: %.4f' % (sigdBack, sigdGM, sigdWM))
print('ME_Back: %.4f \t ME_GM: %.4f \t ME_WM: %.4f' % (marginErrorBack, marginErrorGM, marginErrorWM))
print('n_dBack: %.4f \t n_dGM: %.4f \t n_dWM: %.4f' % (n_dBack, n_dGM, n_dWM))
print('#################### IoU ####################')
print('IoUBack'); print(IoUBack); print('')
print('IoUGM'); print(IoUGM); print('')
print('IoUWM'); print(IoUWM); print('')
print('avgIoUBack: %.4f \t avgIoUGM: %.4f \t avgIoUWM: %.4f' % (avgIoUBack, avgIoUGM, avgIoUWM))
print('sigIoUBack: %.4f \t sigIoUGM: %.4f \t sigIoUWM: %.4f' % (sigIoUBack, sigIoUGM, sigIoUWM))
print('ME_IoUBack: %.4f \t ME_IoUGM: %.4f \t ME_IoUWM: %.4f' % (marginErrorIoUBack, marginErrorIoUGM, marginErrorIoUWM))
print('n_IoUBack: %.4f \t n_IoUGM: %.4f \t n_IoUWM: %.4f' % (n_IoUBack, n_IoUGM, n_IoUWM))
print('#################### F1 ####################')
print('F1Back'); print(F1Back); print('')
print('F1GM'); print(F1GM); print('')
print('F1WM'); print(F1WM); print('')
print('avgF1Back: %.4f \t avgF1GM: %.4f \t avgF1WM: %.4f' % (avgF1Back, avgF1GM, avgF1WM))
print('sigF1Back: %.4f \t sigF1GM: %.4f \t sigF1WM: %.4f' % (sigF1Back, sigF1GM, sigF1WM))
print('ME_F1Back: %.4f \t ME_F1GM: %.4f \t ME_F1WM: %.4f' % (marginErrorF1Back, marginErrorF1GM, marginErrorF1WM))
print('n_F1Back: %.4f \t n_F1GM: %.4f \t n_F1WM: %.4f' % (n_F1Back, n_F1GM, n_F1WM))

alpha: 0.0500
z_alpha: 1.9600
#################### Pixel Accuracy ####################
dBack
[0.69284523 0.89097502 0.88849591 0.8919946  0.9058112  0.95208189
 0.89379535 0.914146   0.91957206 0.93083263 0.93454218 0.77748227
 0.91216647 0.8797433  0.92658354 0.92392618 0.92766204 0.82875509]

dGM
[0.62282831 0.63351721 0.54768519 0.76691744 0.8205319  0.77114374
 0.67638124 0.79694117 0.83492859 0.80878018 0.53117927 0.61576775
 0.81911091 0.4719277  0.69335718 0.79789095 0.764389   0.74348456]

dWM
[0.83011624 0.71546912 0.62348191 0.85150463 0.87108898 0.7835961
 0.75096709 0.80378479 0.86438987 0.8391695  0.5724537  0.80883474
 0.8424781  0.52046906 0.70880682 0.80770962 0.77312671 0.86731821]

avgdBack: 0.8884 	 avgdGM: 0.7065 	 avgdWM: 0.7686
sigdBack: 0.0041 	 sigdGM: 0.0126 	 sigdWM: 0.0108
ME_Back: 0.0500 	 ME_GM: 0.0500 	 ME_WM: 0.0500
n_dBack: 6.2344 	 n_dGM: 19.3739 	 n_dWM: 16.5375
#################### IoU ####################
IoUBack
[0.40882198 0.7940982  0.69877258 0.7

In [0]:
# Faulty Implementation (DON'T USE)
import scipy.stats

# Significance threshold (acceptable Type I error)
alpha = 0.05
# beta = 1 − power (acceptable Type II error)
# beta = 0.1
# Calculate t values
t_alpha = scipy.stats.norm(0, 1).ppf(1 - alpha)   # one-tailed
# t_beta = scipy.stats.norm(0, 1).ppf(beta)   # one-tailed

#################### Pixel Accuracy ####################
# Average per-image difference
avgdBack = np.mean(dBack)
avgdGM = np.mean(dGM)
avgdWM = np.mean(dWM)
# Estimate Per-image difference variance
sigdBack = np.sum(np.square(dBack - avgdBack)) / (len(image_names) - 1)
sigdGM = np.sum(np.square(dGM - avgdGM)) / (len(image_names) - 1)
sigdWM = np.sum(np.square(dWM - avgdWM)) / (len(image_names) - 1)

# Specify intended accuracies
goal_dBack = 0.9; goal_dGM = 0.9; goal_dWM = 0.9
null_dBack = 0.85; null_dGM = 0.85; null_dWM = 0.85
# Estimate sample size
# n_dBack = ((t_alpha - t_beta) * np.sqrt(sigdBack) / (goal_dBack - avgdBack)) ** 2
# n_dGM = ((t_alpha - t_beta) * np.sqrt(sigdGM) / (goal_dGM - avgdGM)) ** 2
# n_dWM = ((t_alpha - t_beta) * np.sqrt(sigdWM) / (goal_dWM - avgdWM)) ** 2
n_dBack = ((t_alpha - t_beta) * np.sqrt(sigdBack) / (avgdBack - null_dBack)) ** 2
n_dGM = ((t_alpha - t_beta) * np.sqrt(sigdGM) / (avgdGM - null_dGM)) ** 2
n_dWM = ((t_alpha - t_beta) * np.sqrt(sigdWM) / (avgdWM - null_dWM)) ** 2

#################### IoU and F1 ####################
# Average IoU and F1
avgIoUBack = np.mean(IoUBack)
avgIoUGM = np.mean(IoUGM)
avgIoUWM = np.mean(IoUWM)
avgF1Back = np.mean(F1Back)
avgF1GM = np.mean(F1GM)
avgF1WM = np.mean(F1WM)
# Estimate IoU and F1 variances
sigIoUBack = np.sum(np.square(IoUBack - avgIoUBack)) / (len(image_names) - 1)
sigIoUGM = np.sum(np.square(IoUGM - avgIoUGM)) / (len(image_names) - 1)
sigIoUWM = np.sum(np.square(IoUWM - avgIoUWM)) / (len(image_names) - 1)
sigF1Back = np.sum(np.square(F1Back - avgF1Back)) / (len(image_names) - 1)
sigF1GM = np.sum(np.square(F1GM - avgF1GM)) / (len(image_names) - 1)
sigF1WM = np.sum(np.square(F1WM - avgF1WM)) / (len(image_names) - 1)

# Specify intended accuracies
goal_IoUBack = 0.9; goal_IoUGM = 0.9; goal_IoUWM = 0.9
goal_F1Back = 0.9;  goal_F1GM = 0.9;  goal_F1WM = 0.9
null_IoUBack = 0.5; null_IoUGM = 0.5; null_IoUWM = 0.5
null_F1Back = 0.5; null_F1GM = 0.5; null_F1WM = 0.5
# Estimate sample size
# n_IoUBack = ((t_alpha - t_beta) * np.sqrt(sigIoUBack) / (goal_IoUBack - avgIoUBack)) ** 2
# n_IoUGM = ((t_alpha - t_beta) * np.sqrt(sigIoUGM) / (goal_IoUGM - avgIoUGM)) ** 2
# n_IoUWM = ((t_alpha - t_beta) * np.sqrt(sigIoUWM) / (goal_IoUWM - avgIoUWM)) ** 2
# n_F1Back = ((t_alpha - t_beta) * np.sqrt(sigF1Back) / (goal_F1Back - avgF1Back)) ** 2
# n_F1GM = ((t_alpha - t_beta) * np.sqrt(sigF1GM) / (goal_F1GM - avgF1GM)) ** 2
# n_F1WM = ((t_alpha - t_beta) * np.sqrt(sigF1WM) / (goal_F1WM - avgF1WM)) ** 2
n_IoUBack = ((t_alpha - t_beta) * np.sqrt(sigIoUBack) / (avgIoUBack - null_IoUBack)) ** 2
n_IoUGM = ((t_alpha - t_beta) * np.sqrt(sigIoUGM) / (avgIoUGM - null_IoUGM)) ** 2
n_IoUWM = ((t_alpha - t_beta) * np.sqrt(sigIoUWM) / (avgIoUWM - null_IoUWM)) ** 2
n_F1Back = ((t_alpha - t_beta) * np.sqrt(sigF1Back) / (avgF1Back - null_F1Back)) ** 2
n_F1GM = ((t_alpha - t_beta) * np.sqrt(sigF1GM) / (avgF1GM - null_F1GM)) ** 2
n_F1WM = ((t_alpha - t_beta) * np.sqrt(sigF1WM) / (avgF1WM - null_F1WM)) ** 2

print('alpha: %.4f \t beta:%.4f' % (alpha, beta))
print('t_alpha: %.4f \t t_beta:%.4f' % (t_alpha, t_beta))
print('#################### Pixel Accuracy ####################')
print('dBack'); print(dBack); print('')
print('dGM'); print(dGM); print('')
print('dWM'); print(dWM); print('')
print('avgdBack: %.4f \t avgdGM: %.4f \t avgdWM: %.4f' % (avgdBack, avgdGM, avgdWM))
print('sigdBack: %.4f \t sigdGM: %.4f \t sigdWM: %.4f' % (sigdBack, sigdGM, sigdWM))
print('goal_dBack: %.4f \t goal_dGM: %.4f \t goal_dWM: %.4f' % (goal_dBack, goal_dGM, goal_dWM))
print('n_dBack: %.4f \t n_dGM: %.4f \t n_dWM: %.4f' % (n_dBack, n_dGM, n_dWM))
print('#################### IoU ####################')
print('IoUBack'); print(IoUBack); print('')
print('IoUGM'); print(IoUGM); print('')
print('IoUWM'); print(IoUWM); print('')
print('avgIoUBack: %.4f \t avgIoUGM: %.4f \t avgIoUWM: %.4f' % (avgIoUBack, avgIoUGM, avgIoUWM))
print('sigIoUBack: %.4f \t sigIoUGM: %.4f \t sigIoUWM: %.4f' % (sigIoUBack, sigIoUGM, sigIoUWM))
print('goal_IoUBack: %.4f \t goal_IoUGM: %.4f \t goal_IoUWM: %.4f' % (goal_IoUBack, goal_IoUGM, goal_IoUWM))
print('n_IoUBack: %.4f \t n_IoUGM: %.4f \t n_IoUWM: %.4f' % (n_IoUBack, n_IoUGM, n_IoUWM))
print('#################### F1 ####################')
print('F1Back'); print(F1Back); print('')
print('F1GM'); print(F1GM); print('')
print('F1WM'); print(F1WM); print('')
print('avgF1Back: %.4f \t avgF1GM: %.4f \t avgF1WM: %.4f' % (avgF1Back, avgF1GM, avgF1WM))
print('sigF1Back: %.4f \t sigF1GM: %.4f \t sigF1WM: %.4f' % (sigF1Back, sigF1GM, sigF1WM))
print('goal_F1Back: %.4f \t goal_F1GM: %.4f \t goal_F1WM: %.4f' % (goal_F1Back, goal_F1GM, goal_F1WM))
print('n_F1Back: %.4f \t n_F1GM: %.4f \t n_F1WM: %.4f' % (n_F1Back, n_F1GM, n_F1WM))

alpha: 0.0500 	 beta:0.1000
t_alpha: 1.6449 	 t_beta:-1.2816
#################### Pixel Accuracy ####################
dBack
[0.69284523 0.89097502 0.88849591 0.8919946  0.9058112  0.95208189
 0.89379535 0.914146   0.91957206 0.93083263 0.93454218 0.77748227
 0.91216647 0.8797433  0.92658354 0.92392618 0.92766204 0.82875509]

dGM
[0.62282831 0.63351721 0.54768519 0.76691744 0.8205319  0.77114374
 0.67638124 0.79694117 0.83492859 0.80878018 0.53117927 0.61576775
 0.81911091 0.4719277  0.69335718 0.79789095 0.764389   0.74348456]

dWM
[0.83011624 0.71546912 0.62348191 0.85150463 0.87108898 0.7835961
 0.75096709 0.80378479 0.86438987 0.8391695  0.5724537  0.80883474
 0.8424781  0.52046906 0.70880682 0.80770962 0.77312671 0.86731821]

avgdBack: 0.8884 	 avgdGM: 0.7065 	 avgdWM: 0.7686
sigdBack: 0.0041 	 sigdGM: 0.0126 	 sigdWM: 0.0108
goal_dBack: 0.9000 	 goal_dGM: 0.9000 	 goal_dWM: 0.9000
n_dBack: 23.5495 	 n_dGM: 5.2426 	 n_dWM: 13.9096
#################### IoU ####################
IoUBa

#### Power Analysis (Two algorithms)
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5666910/

In [0]:
# Per-image accuracy difference of a simulated image
dk = np.zeros(len(image_names))

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
  # Filename for testing and ground truth images
  test_grey_img_name = imagename + "-Grey.png"
  test_white_img_name = imagename + "-White.png"
  truth_grey_img_name = imagename + "-Grey.png"
  truth_white_img_name = imagename + "-White.png"

  ##### Loading Grey Matter Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + test_grey_img_name))
  truth_img = Image.open(TRUTH_DIR + truth_grey_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img) * 1

  # Check dimensions
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' grey dimension WRONG!!!')

  # Update tqdm descriptions
  t.set_description_str("Image {}, Test ({}), Ground Truth ({})".format(
      imagename, str(test_img.shape), str(truth_img.shape)) )
  t.refresh()
  t.write("", end=' ')

  # Clip test_img to 1
  test_img[test_img>=1] = 1


  # Per-voxel accuracy difference of a simulated voxel: |bk_i - lk_i|
  dk_i = np.abs(test_img - truth_img)

  ##### Loading White Matter Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + test_white_img_name))
  truth_img = Image.open(TRUTH_DIR + truth_white_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img) * 1

  # Check dimensions
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' white dimension WRONG!!!')

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Per-voxel accuracy difference of a simulated voxel: |bk_i - lk_i| - |ak_i - lk_i|
  dk_i -= np.abs(test_img - truth_img)

  # Calculate per-image accuracy difference dk
  dk[i] = np.sum(dk_i) / (width * height)

  t.update()
t.close()

print('\nDone!')

In [0]:
import scipy.stats

# Significance threshold (acceptable Type I error)
alpha = 0.05

# beta = 1 − power (acceptable Type II error)
beta = 0.2

# Minimum difference to detect with specified power
delta_MDD = 0.03

# Estimate the population average accuracy difference
delta_hat = np.sum(dk) / len(image_names)

# Estimate per-image accuracy difference variance (Eqn. 13)
sig_D_hat = np.sum(np.square(dk - delta_hat)) / (len(image_names) - 1)

# Estimate the required sample size
t_alpha = scipy.stats.norm(0, 1).ppf(1 - alpha/2)   # two-tailed
t_beta = scipy.stats.norm(0, 1).ppf(1 - beta)   # one-tailed
n = np.square(np.sqrt(sig_D_hat) * (t_alpha + t_beta)) / (delta_MDD**2)

print('dk')
print(dk)
print('\n')
print('delta_hat: %.4f\n' % (delta_hat))
print('sig_D_hat: %.4f\n' % (sig_D_hat))
print('n: %.4f\n' % (n))

#### Power Analysis (Two algorithms psi)

In [0]:
# probability of disagreement
psi = np.zeros(len(image_names))

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
  # Filename for testing images
  test_grey_img_name = imagename + "-Grey.png"
  test_white_img_name = imagename + "-White.png"

  ##### Loading Grey Matter Mask #####
  # Load images
  test_grey_img = np.array(Image.open(TEST_DIR + test_grey_img_name), dtype='uint')

  ##### Loading White Matter Mask #####
  # Load images
  test_white_img = np.array(Image.open(TEST_DIR + test_white_img_name), dtype='uint')
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)

  # Check dimensions
  if not (test_grey_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_grey_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' grey dimension WRONG!!!')
  if not (test_white_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_white_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' white dimension WRONG!!!')

  # Update tqdm descriptions
  t.set_description_str("Image {}, Test Grey ({}), Test White ({})".format(
      imagename, str(test_grey_img.shape), str(test_white_img.shape)) )
  t.refresh()
  t.write("", end=' ')

  # Clip test_img to 1
  test_grey_img[test_grey_img>=1] = 1
  test_white_img[test_white_img>=1] = 1
  print(np.any(test_grey_img == 0))

  # Per-image probability of disagreement: |ak_i - bk_i|
  psi[i] = np.sum(np.abs(test_grey_img - test_white_img)) / (width * height)

  t.update()
t.close()

print('\nDone!')
print(psi)
psi_avg = np.sum(psi) / len(image_names)
print('psi_avg: %.4e' % psi_avg)